In [1]:
import nltk
import collections
import re
from nltk.corpus import brown

In [2]:
text=brown.words()
print(text)
print(len(text))

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
1161192


In [3]:
#先验，统计每个词出现的概率
def prior(cwords):
    model = collections.defaultdict(lambda:1)
    for f in cwords:
        model[f]+=1
    return model

cwords=text
nwords = prior(cwords)

In [4]:
#类条件概率
alpha = 'abcdefghijklmnopqrstuvwxyz'
#一步调整
def version1(word):
    n = len(word)
    add_a_char = [word[0:i] + c + word[i:] for i in range(n+1) for c in alpha]
    delete_a_char = [word[0:i] + word[i+1:] for i in range(n)]
    revise_a_char = [word[0:i] + c + word[i+1:] for i in range(n) for c in alpha]
    swap_adjacent_two_chars = [word[0:i] + word[i+1]+ word[i]+ word[i+2:] for i in range(n-1)] 
    return set( add_a_char + delete_a_char +
               revise_a_char +  swap_adjacent_two_chars)

In [5]:
#两步调整           
def version2(word):
    return set(e2 for e1 in version1(word) for e2 in version1(e1))


#朴素贝叶斯分类器

def identify(words):
    return set(w for w in words if w in nwords)

def getMax(wanteds):
    threewanteds=[]
    maxword = max(wanteds,key=lambda w : nwords[w])
    threewanteds.append(maxword)
    wanteds.remove(maxword)
    if len(wanteds)>0:
        maxword = max(wanteds,key=lambda w : nwords[w])
        threewanteds.append(maxword)
        wanteds.remove(maxword)
        if len(wanteds)>0:
            maxword = max(wanteds,key=lambda w : nwords[w])
            threewanteds.append(maxword)   
    return threewanteds

In [6]:
def bayesClassifier(word):
    #如果字典中有输入的单词，直接返回
    if identify([word]):
        return 'found: '+ word
    #一步调整
    wanteds = identify(version1(word)) 
    if len(wanteds)>0:
        return getMax(wanteds)
    #两步调整
    wanteds = identify(version2(word))
    if len(wanteds)>0:
        return getMax(wanteds)
    #不再修正，直接提示这个单词不在当前的词典中
    else:    
        return [word + ' not found in dictionary!' ]

In [7]:
print(bayesClassifier('kami'))

['same', 'came', 'name']


In [8]:
print(bayesClassifier('jia'))

['via', 'jig']
